# Imports

In [3]:
import sys
import os
from pathlib import Path
from datetime import datetime
import glob

# Change to project root directory
project_root = '/Users/lucasmuller/Desktop/Githubg/Rawls_v3'
os.chdir(project_root)
sys.path.append(project_root)

from utils.experiment_runner import generate_random_config, generate_and_save_configs, run_experiment, run_experiments_parallel



# Parameter Space


In [13]:
models = [
    "x-ai/grok-4",
    "google/gemini-2.5-flash",
    "google/gemini-2.5-pro",
    "meta/llama/llama-4-maverick",
    "anthropic/claude-sonnet-4",
    "openai/gpt-4.1-mini",
    "openai/gpt-4.1",
    "deepseek/deepseek-chat-v3-0324",
    "qwen/qwen3-coder",
    "moonshotai/kimi-k2"
]

In [14]:

def generate_random_config(
      num_agents: int = 3,
      num_rounds: int = 20,
      personality: str = "You are an american college student.",
      models: Union[str, List[str]] = models,
      temperature: Union[float, tuple] = (0.0, 1.0),
      memory_limit: int = 50000,
      reasoning_enabled: bool = True,
      utility_model: str = "gpt-4.1-mini",
      language: str = "English",
      distribution_range_phase1: List[float] = [0.5, 2.0],
      distribution_range_phase2: List[float] = [0.5, 2.0]
  ) -> Dict[str, Any]:
      """
      Generate a random configuration for the Frohlich Experiment.
      
      Parameters:
      - num_agents: Number of participant agents
      - num_rounds: Number of Phase 2 rounds
      - personality: Single personality description used for all agents
      - models: Single model string or list to randomly select from
      - temperature: Fixed temperature (float) or range (tuple) for random selection
      - memory_limit: Memory character limit for agents
      - reasoning_enabled: Whether agents use reasoning
      - utility_model: Model for utility agent
      - language: Language for prompts ("English", "Spanish", "Mandarin")
      - distribution_range_phase1/2: Income distribution multiplier ranges
      
      Returns:
      - Dictionary representing the experiment configuration
      """

      # Generate agents
      agents = []
      for i in range(num_agents):
          # Generate agent name as Agent_1, Agent_2, etc.
          agent_name = f"Agent_{i + 1}"

          # Select model
          if isinstance(models, list):
              selected_model = random.choice(models)
          else:
              selected_model = models

          # Select temperature
          if isinstance(temperature, tuple) and len(temperature) == 2:
              selected_temp = round(random.uniform(temperature[0], temperature[1]), 2)
          else:
              selected_temp = float(temperature)

          agent_config = {
              "name": agent_name,
              "personality": personality,
              "model": selected_model,
              "temperature": selected_temp,
              "memory_character_limit": memory_limit,
              "reasoning_enabled": reasoning_enabled
          }
          agents.append(agent_config)

      # Build complete configuration
      config = {
          "language": language,
          "agents": agents,
          "utility_agent_model": utility_model,
          "phase2_rounds": num_rounds,
          "distribution_range_phase1": distribution_range_phase1,
          "distribution_range_phase2": distribution_range_phase2
      }

      return config

def generate_and_save_configs(
      num_configs: int = 10,
      save_path: str = "hypothesis_2_&_4/configs/condition_1",
      **config_params
  ):
      """
      Generate multiple random configurations and save them as YAML files.
      
      Parameters:
      - num_configs: Number of configurations to generate
      - save_path: Path to save the configuration files
      - **config_params: Parameters to pass to generate_random_config()
      
      Returns:
      - List of generated configurations
      """

      # Create directory if it doesn't exist
      os.makedirs(save_path, exist_ok=True)

      configs = []
      for i in range(num_configs):
          # Generate random config
          config = generate_random_config(**config_params)
          configs.append(config)

          # Save to YAML file
          filename = f"config_{i+1:02d}.yaml"
          filepath = os.path.join(save_path, filename)

          with open(filepath, 'w') as f:
              yaml.dump(config, f, default_flow_style=False, indent=2)

          print(f"Saved {filename}")

      print(f"Generated and saved {num_configs} configurations in {save_path}")
      return configs

  # Example usage:


# 1. Condition

## Generating Configs

In [ ]:
# Generate configs 1. 
# Commenteed out to avoid running if accidentally clicked run all 
#configs = generate_and_save_configs(
    num_configs=10,
    num_agents=5,
    personality="You are an american college student.",
    models=models,
    temperature=(0),
    num_rounds=20,
 )

Saved config_01.yaml
Saved config_02.yaml
Saved config_03.yaml
Saved config_04.yaml
Saved config_05.yaml
Saved config_06.yaml
Saved config_07.yaml
Saved config_08.yaml
Saved config_09.yaml
Saved config_10.yaml
Generated and saved 10 configurations in hypothesis_2_&_4/configs/condition_1


## Running Configs

In [ ]:
# Get all config files from condition_1
config_dir = "hypothesis_2_&_4/configs/condition_1"
config_files = sorted(glob.glob(f"{config_dir}/config_*.yaml"))

print(f"Found {len(config_files)} configuration files:")
for i, config_file in enumerate(config_files, 1):
    config_name = Path(config_file).name
    print(f"  {i:2d}. {config_name}")

# Display the total
print(f"\nTotal configurations to run: {len(config_files)}")

In [ ]:
# Set up execution parameters
MAX_PARALLEL = 5  # Number of experiments to run concurrently
OUTPUT_DIR = "hypothesis_2_&_4/logs"  # Directory to save results

# Create logs directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Starting parallel execution of {len(config_files)} experiments")
print(f"Maximum parallel experiments: {MAX_PARALLEL}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n" + "="*80 + "\n")

# Run the experiments
try:
    results = run_experiments_parallel(
        config_files=config_files,
        max_parallel=MAX_PARALLEL,
        output_dir=OUTPUT_DIR,
        verbose=True
    )
    
    print("\n" + "="*80)
    print(f"Parallel execution completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
except Exception as e:
    print(f"\nParallel execution failed with error: {e}")
    import traceback
    traceback.print_exc()
    results = None

# Example: Run Config_01 from Condition 1

In [16]:
# Load config_01.yaml
import yaml
from datetime import datetime

config_path = 'hypothesis_2_&_4/configs/condition_1/config_01.yaml'

with open(config_path, 'r') as f:
    config_dict = yaml.safe_load(f)

print("Loaded configuration:")
print(f"- Language: {config_dict['language']}")
print(f"- Number of agents: {len(config_dict['agents'])}")
print(f"- Phase 2 rounds: {config_dict['phase2_rounds']}")
print(f"- Utility agent model: {config_dict['utility_agent_model']}")
print("\nAgent details:")
for agent in config_dict['agents']:
    print(f"  - {agent['name']}: {agent['model']} (temp={agent['temperature']})")

Loaded configuration:
- Language: English
- Number of agents: 5
- Phase 2 rounds: 20
- Utility agent model: gpt-4.1-mini

Agent details:
  - Agent_1: google/gemini-2.5-pro (temp=0.0)
  - Agent_2: google/gemini-2.5-pro (temp=0.0)
  - Agent_3: openai/gpt-4.1-mini (temp=0.0)
  - Agent_4: anthropic/claude-sonnet-4 (temp=0.0)
  - Agent_5: anthropic/claude-sonnet-4 (temp=0.0)


In [4]:
# Create output path with timestamp in logs folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f"hypothesis_2_&_4/logs/config_01_results_{timestamp}.json"

print(f"Running experiment with config_01...")
print(f"Results will be saved to: {output_path}")
print("\n" + "="*60)

# Run the experiment
try:
    results = run_experiment(
        config_dict=config_dict,
        output_path=output_path,
        verbose=True
    )
    print("\n" + "="*60)
    print("EXPERIMENT COMPLETED SUCCESSFULLY!")
    print(f"Results saved to: {output_path}")
    
except Exception as e:
    print(f"\nExperiment failed with error: {e}")
    import traceback
    traceback.print_exc()

Running experiment with config_01...
Results will be saved to: hypothesis_2_&_4/logs/config_01_results_20250810_103542.json


Experiment failed with error: name 'config_dict' is not defined


Traceback (most recent call last):
  File "/var/folders/wf/_h59fnv53s7476fhw5sn5smh0000gn/T/ipykernel_65546/2338803112.py", line 12, in <module>
    config_dict=config_dict,
                ^^^^^^^^^^^
NameError: name 'config_dict' is not defined


In [5]:
# Get all config files from condition_1
config_dir = "hypothesis_2_&_4/configs/condition_1"
config_files = sorted(glob.glob(f"{config_dir}/config_*.yaml"))

print(f"Found {len(config_files)} configuration files:")
for i, config_file in enumerate(config_files, 1):
    config_name = Path(config_file).name
    print(f"  {i:2d}. {config_name}")

# Display the total
print(f"\nTotal configurations to run: {len(config_files)}")

Found 10 configuration files:
   1. config_01.yaml
   2. config_02.yaml
   3. config_03.yaml
   4. config_04.yaml
   5. config_05.yaml
   6. config_06.yaml
   7. config_07.yaml
   8. config_08.yaml
   9. config_09.yaml
  10. config_10.yaml

Total configurations to run: 10


In [ ]:
# Set up execution parameters
MAX_PARALLEL = 5  # Number of experiments to run concurrently
OUTPUT_DIR = "hypothesis_2_&_4/logs"  # Directory to save results

# Create logs directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Starting parallel execution of {len(config_files)} experiments")
print(f"Maximum parallel experiments: {MAX_PARALLEL}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n" + "="*80 + "\n")

# Run the experiments
try:
    results = run_experiments_parallel(
        config_files=config_files,
        max_parallel=MAX_PARALLEL,
        output_dir=OUTPUT_DIR,
        verbose=True
    )
    
    print("\n" + "="*80)
    print(f"Parallel execution completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
except Exception as e:
    print(f"\nParallel execution failed with error: {e}")
    import traceback
    traceback.print_exc()
    results = None

Starting parallel execution of 10 experiments
Maximum parallel experiments: 5
Output directory: hypothesis_2_&_4/logs
Start time: 2025-08-10 10:36:18


2025-08-10 10:36:18,695 - utils.experiment_runner - INFO - Loaded 10 configurations
2025-08-10 10:36:18,695 - utils.experiment_runner - INFO - Running with max 5 parallel experiments
2025-08-10 10:36:18,696 - utils.experiment_runner - INFO - Starting parallel experiment execution...
2025-08-10 10:36:18,696 - utils.experiment_runner - INFO - Starting experiment: config_01
2025-08-10 10:36:18,702 - utils.language_manager - INFO - Language set to: English
2025-08-10 10:36:18,704 - core.experiment_manager - INFO - Created participant: Agent_1 (google/gemini-2.5-pro, temp=0.0)
2025-08-10 10:36:18,704 - core.experiment_manager - INFO - Created participant: Agent_2 (google/gemini-2.5-pro, temp=0.0)
2025-08-10 10:36:18,705 - core.experiment_manager - INFO - Created participant: Agent_3 (openai/gpt-4.1-mini, temp=0.0)
2025-08-10 10:36:18,705 - c

10:36:18 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= google/gemini-2.5-pro; provider = openrouter


2025-08-10 10:36:18,751 - LiteLLM - INFO - 
LiteLLM completion() model= google/gemini-2.5-pro; provider = openrouter


10:36:18 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= google/gemini-2.5-pro; provider = openrouter


2025-08-10 10:36:18,752 - LiteLLM - INFO - 
LiteLLM completion() model= google/gemini-2.5-pro; provider = openrouter


...

2025-08-10 10:57:46,593 - LiteLLM - INFO - 
LiteLLM completion() model= google/gemini-2.5-pro; provider = openrouter
2025-08-10 10:57:49,510 - httpx - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-10 10:57:50,449 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"


10:57:54 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= deepseek/deepseek-chat-v3-0324; provider = openrouter


2025-08-10 10:57:54,047 - LiteLLM - INFO - 
LiteLLM completion() model= deepseek/deepseek-chat-v3-0324; provider = openrouter
2025-08-10 10:57:55,308 - httpx - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-10 10:57:55,967 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"


KeyboardInterrupt: 

2025-08-10 10:58:06,975 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"


## Quick Results Summary